In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime
from dateutil.parser import parse
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from pprint import pprint
import undetected_chromedriver as uc

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("WAN HAI")

In [4]:
gagal = []

In [5]:
hasil_akhir = []
# accesing web
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
url = "https://www.wanhai.com/views/cargo_track_v2/tracking_query.xhtml"
driver.get(url)

for i, bls in enumerate(bl_list):
    try: 
        # input BL number
        input_bl = driver.find_element(By. XPATH,'/html/body/div[2]/div[3]/div[3]/div/div/form/table[1]/tbody/tr[2]/td[2]/input[1]')
        input_bl.clear()
        input_bl.send_keys(bl_list[i])

        # click querry
        querry_button = driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[3]/div/div/form/table[2]/tbody/tr/td[2]/input')
        querry_button.click()

        # switch to new window
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)

        # BL Data
        more_bl_data = driver.find_element(By.LINK_TEXT, 'B/L Data')
        more_bl_data.click()

        # switch to new window
        driver.switch_to.window(driver.window_handles[2])
        time.sleep(2)

        # find how many container in 1 bl
        soup = BeautifulSoup(driver.page_source, 'lxml') 
        table = soup.find_all('table', {'class':'tbl-list'})
        list_of_containers = []
        for tag in table:
            for a, href  in enumerate(tag.find_all('a')):
                list_of_containers.append(href.text.replace(" ","").replace("\n",""))

        places=[]
        origin = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/form/table[5]/tbody/tr[2]/td[1]')
        destination = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/form/table[5]/tbody/tr[5]/td[1]')
        places.append(origin.text)
        places.append(destination.text)
        # iterating data scraping for every container
        for container in list_of_containers:

            # click container
            more_bl_data = driver.find_element(By.LINK_TEXT, container)
            more_bl_data.click()

            # switch to new window
            driver.switch_to.window(driver.window_handles[3])
            time.sleep(3)

            # scraping container data
            soup2 = BeautifulSoup(driver.page_source, 'lxml') 

            # finding data and adding to current dict
            current_dict = {}
            current_dict.update({"Liners": "WAN HAI"})
            current_dict.update({"BL Number": bls})
            current_dict.update({"Container Number": container})
            current_dict.update({"From": places[0]})
            current_dict.update({"To": places[1]})
            table_data = soup2.find_all('table', {'class':'tbl-list'})

            for tag2 in table_data:
                for d, data in enumerate(tag2.find_all('td')):
                    if is_date(data.text):
                        case_milestone = {tag2.find_all('td')[d+1].text.replace("\n","").strip():tag2.find_all('td')[d].text.replace("\n","").strip()[s]}
                        current_dict.update(case_milestone)
            
            hasil_akhir.append(current_dict)
            # closing windows dan switch to handle 2
            driver.close()
            driver.switch_to.window(driver.window_handles[2])
            time.sleep(2)
        driver.close()
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

In [6]:
gagal

[]

In [7]:
hasil_akhir2 = hasil_akhir

In [8]:
hasil_akhir_fixed = []
for filter_dict in hasil_akhir2:
    replacement_mapping = {
        "Laden Ctnr Loaded On Board Vessel.(Mother Vessel,Fixed Slot Charter/Joint Venture Vessel)": 'ATD',
        "Full Container(Fcl) Discharged From Vessel Or Gate In To Pier/Terminal": 'ATA',
        "Full Container Withdrawn By Consignee From Pier/Terminal": 'Container Release',
        "Empty Container Discharged From Vessel Or Gate In To Pier/Terminal/Off-Dock Depot" : 'Container Return'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    hasil_akhir_fixed.append(updated_dict)

In [9]:
hasil_akhir_fixed

[{'Liners': 'WAN HAI',
  'BL Number': '087DA00357',
  'Container Number': 'WHSU5139551',
  'From': 'JAKARTA (ID)',
  'To': 'CEBU (PH)',
  'Container Return (Empty Available)': '2023/09/16',
  'Container Release': '2023/08/23',
  'ATA': '2023/08/14',
  'ATD': '2023/07/31',
  'Laden Or Empty Transit Container Discharged From Own Or Feeder Vessel At Pier/Terminal Awaiting  For': '2023/08/08',
  'Inbound/Outbound Laden Ctnr Move From One Pier/Terminal To Another One By Trucker/Rail.': '2023/08/10',
  'Outbound Full Container Gate In To Peir/Terminal(Fcl)': '2023/07/28',
  'Empty Container Released To Shipper From Pier/Terminal/Off-Dock Depot': '2023/07/26'},
 {'Liners': 'WAN HAI',
  'BL Number': '087DA00395',
  'Container Number': 'TCKU7869527',
  'From': 'JAKARTA (ID)',
  'To': 'DA NANG (PORT) (VN)',
  'ATA': '2023/09/10',
  'ATD': '2023/08/20',
  'Laden Or Empty Transit Container Discharged From Own Or Feeder Vessel At Pier/Terminal Awaiting  For': '2023/08/28',
  'Inbound/Outbound Laden

In [11]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(hasil_akhir_fixed)

Today's Collection Name ===>  all_tracking_Sep-18-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [19]:
import pandas as pd
df = pd.DataFrame(hasil_akhir_fixed)

excel_file_path = 'export excel/WANHAI.xlsx'

df.to_excel(excel_file_path, index=False)